In [8]:
import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence

# Function to convert from milliseconds to SRT time format
def milliseconds_to_srt_time(ms):
    hours, ms = divmod(ms, 3600000)
    minutes, ms = divmod(ms, 60000)
    seconds, ms = divmod(ms, 1000)
    return f'{int(hours):02}:{int(minutes):02}:{int(seconds):02},{int(ms):03}'

# Load your audio file (change 'audio.wav' to your file's name)
audio_file = "audio.wav"
audio = AudioSegment.from_wav(audio_file)

# Split audio into chunks based on silence
chunks = split_on_silence(audio, 
                          min_silence_len=300,
                          silence_thresh=audio.dBFS-14,
                          keep_silence=300)

# Initialize the recognizer
r = sr.Recognizer()

# Prepare to write the SRT file
srt_file_content = ""
chunk_start_time = 0

for i, chunk in enumerate(chunks):
    # Export the audio chunk to a wav file
    chunk_filename = f"chunk{i}.wav"
    chunk.export(chunk_filename, format="wav")
    
    # Use the recognizer to transcribe the chunk
    with sr.AudioFile(chunk_filename) as source:
        chunk_audio_data = r.record(source)
        try:
            # Recognize speech using Google's Web Speech API
            chunk_text = r.recognize_google(chunk_audio_data, language='hi-IN')
            
            # Calculate the end time for this chunk
            chunk_end_time = chunk_start_time + len(chunk)
            
            # Append the transcription and timings to the SRT file content
            srt_file_content += f"{i+1}\n"
            srt_file_content += f"{milliseconds_to_srt_time(chunk_start_time)} --> {milliseconds_to_srt_time(chunk_end_time)}\n"
            srt_file_content += f"{chunk_text}\n\n"
            
            # Update the start time for the next chunk
            chunk_start_time = chunk_end_time + 1
        except sr.UnknownValueError:
            print(f"Chunk {i} could not be transcribed.")
        except sr.RequestError as e:
            print(f"Could not request results; {e}")

# Write the SRT file
with open("output.txt", "w") as file:
    file.write(srt_file_content)

print("SRT file created successfully.")


UnicodeEncodeError: 'charmap' codec can't encode characters in position 34-35: character maps to <undefined>

In [9]:
srt_file_content

'1\n00:00:00,000 --> 00:00:58,924\nघर में जुड़वा बच्चे हैं पहले के बेडरूम में एक बंक बेड था की अलमारी और एक स्टडी टेबल बहुत मुश्किल से डाला गया दोनों भाई बारी बारी से ही पढ़ सकते थे और दूसरा दीवार के कोने में बैठा था तब दोनों भाई गुस्से से पूरा बेडरूम तोड़ देते हैं तब पापा ने पूरा बेडरूम फिर से बनाया सबसे पहले मजबूत लोहे से दो बेड स्ट्रक्चर बनाएंगे और साथ में मजबूत पिलर भी लगाएंगे जो बेड को गिरने से रोकेगा और बेड को अच्छे स्क्रू से दीवार में कर देंगे उसके ऊपर अच्छे प्लाई का फ्लोरिंग भी करेंगे और एक आरामदायक गद्दा रखेंगे और साथ में तकिया भी रखेंगे बेड के सामने घेरा भी करेंगे ताकि बच्चे सोते वक्त नीचे के रूप में कुछ रक का उपयोग करेंगे जिसमें बच्चों के कपड़े और गंदे मुझे रखेंगे जिसमें बच्चों की सारी पुस्तक रखेंगे और खिड़की के किनारे एक बड़ा टेबल बनाएंगे जिसमें एक तरफ बड़े बेटे का स्टडी एरिया होगा और दूसरे तरफ छोटे बेटे का स्टडी एरिया जिसमें बेडशीट और बच्चों के कपड़े रखेंगे कंप्यूटर रखेंगे यहां अपने पसंद के गेम खेल सके\n\n'

In [7]:
chunks